# Micrograd from Scratch — An Educational Autograd Engine

This notebook builds a tiny autograd engine and a mini MLP on top.

In [ ]:

import math, random
from typing import List, Iterable


In [ ]:

class Value:
    def __init__(self, data, _children=(), _op=''):
        self.data = float(data)
        self.grad = 0.0
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op

    def __repr__(self):
        return f"Value(data={self.data:.6g}, grad={self.grad:.6g}, op='{self._op}')"

    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')
        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad
        out._backward = _backward
        return out

    def __radd__(self, other):
        return self + other

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')
        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward
        return out

    def __rmul__(self, other):
        return self * other

    def __neg__(self):
        return self * -1

    def __sub__(self, other):
        return self + (-other)

    def __rsub__(self, other):
        return other + (-self)

    def __pow__(self, other):
        assert isinstance(other, (int, float))
        out = Value(self.data ** other, (self,), f'**{other}')
        def _backward():
            self.grad += (other * (self.data ** (other - 1))) * out.grad
        out._backward = _backward
        return out

    def __truediv__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        return self * (other ** -1)

    def tanh(self):
        t = (math.exp(2 * self.data) - 1) / (math.exp(2 * self.data) + 1)
        out = Value(t, (self,), 'tanh')
        def _backward():
            self.grad += (1 - t**2) * out.grad
        out._backward = _backward
        return out

    def exp(self):
        out = Value(math.exp(self.data), (self,), 'exp')
        def _backward():
            self.grad += out.data * out.grad
        out._backward = _backward
        return out

    def backward(self):
        topo, visited = [], set()
        def build(v):
            if v not in visited:
                visited.add(v)
                for ch in v._prev:
                    build(ch)
                topo.append(v)
        build(self)
        self.grad = 1.0
        for node in reversed(topo):
            node._backward()


In [ ]:

# Sanity test
a, b, c, f = Value(2.0), Value(-3.0), Value(10.0), Value(-2.0)
L = (a*b + c) * f
L.backward()
print("L:", L.data)
print("dL/da (expected 6):", a.grad)
print("dL/db (expected -4):", b.grad)
print("dL/dc (expected -2):", c.grad)
print("dL/df (expected 4):", f.grad)


In [ ]:

class Neuron:
    def __init__(self, nin: int):
        self.w = [Value(random.uniform(-1, 1)) for _ in range(nin)]
        self.b = Value(0.0)
    def __call__(self, x: List[float]):
        act = sum((wi * xi for wi, xi in zip(self.w, x)), self.b)
        return act.tanh()
    def parameters(self):
        return self.w + [self.b]

class Layer:
    def __init__(self, nin: int, nout: int):
        self.neurons = [Neuron(nin) for _ in range(nout)]
    def __call__(self, x: List[float]):
        outs = [n(x) for n in self.neurons]
        return outs if len(outs) > 1 else outs[0]
    def parameters(self):
        return [p for n in self.neurons for p in n.parameters()]

class MLP:
    def __init__(self, nin: int, nouts: List[int]):
        sz = [nin] + list(nouts)
        self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]
    def __call__(self, x: List[float]):
        for layer in self.layers:
            out = layer(x)
            x = out if isinstance(out, list) else [out]
        return x[0]
    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]


In [ ]:

X = [[2.0, 3.0],[1.0, -1.0],[-1.0, 2.0],[-2.0, -3.0]]
Y = [1.0, -1.0, 1.0, -1.0]

random.seed(42)
model = MLP(2, [4, 4, 1])

def mse_loss(y_pred: Iterable[Value], y_true: Iterable[float]) -> Value:
    loss = Value(0.0)
    for yp, yt in zip(y_pred, y_true):
        loss = loss + (yp - yt)**2
    return loss / len(Y)

lr = 0.05
for step in range(61):
    y_pred = [model(x) for x in X]
    loss = mse_loss(y_pred, Y)
    for p in model.parameters():
        p.grad = 0.0
    loss.backward()
    for p in model.parameters():
        p.data += -lr * p.grad
    if step % 10 == 0:
        print(f"step {step:3d} | loss = {loss.data:.6f}")


In [ ]:

y_pred = [model(x) for x in X]
print("Predictions:", [round(v.data, 3) for v in y_pred])
print("Targets:    ", Y)
